# 最大カバー問題


In [24]:
import pulp
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import japanize_matplotlib

japanize_matplotlib.japanize()
plt.style.use("ggplot")

# パラメータの設定
locations = 20  # 街灯を設置できる場所の数（これがカバーすべき地点の数と同じ）
coverage_range = 2  # 街灯がカバーできる距離
max_lamps = 5  # 設置可能な街灯の最大数
mandatory_points = [0, 5, 10, 15]  # 必ずカバーすべき地点のインデックス

# 位置情報（ランダムに生成）
np.random.seed(0)
positions = np.random.rand(locations, 2) * 10  # 共通の位置情報

# カバー行列の生成
cover = np.zeros((locations, locations), dtype=int)
for i in range(locations):
    for j in range(locations):
        if np.linalg.norm(positions[i] - positions[j]) <= coverage_range:
            cover[i, j] = 1

# PuLPの問題設定
prob = pulp.LpProblem("Street_Light_Placement", pulp.LpMaximize)

# 変数
x = pulp.LpVariable.dicts("x", range(locations), cat="Binary")
y = pulp.LpVariable.dicts("y", range(locations), cat="Binary")

# 目的関数
prob += pulp.lpSum(y[j] for j in range(locations))

# 制約条件
for j in range(locations):
    prob += y[j] <= pulp.lpSum(cover[i][j] * x[i] for i in range(locations))
prob += pulp.lpSum(x[i] for i in range(locations)) <= max_lamps

# 特定の地点を必ずカバーする制約
for j in mandatory_points:
    prob += y[j] == 1

# 求解
prob.solve()

# 結果の表示
lamp_status = [pulp.value(x[i]) for i in range(locations)]
point_covered = [pulp.value(y[j]) for j in range(locations)]

print("Status:", pulp.LpStatus[prob.status])
print("Number of lamps:", sum(lamp_status))
print("Number of covered points:", sum(point_covered))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/shugo/Desktop/Lab/高大連携/code/kodai-renkei/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/l6/wzrvt4j10r97v2dkh7l274fw0000gn/T/7935742f51374427b0ed4e69ddf1f493-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/l6/wzrvt4j10r97v2dkh7l274fw0000gn/T/7935742f51374427b0ed4e69ddf1f493-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 239 RHS
At line 265 BOUNDS
At line 306 ENDATA
Problem MODEL has 25 rows, 40 columns and 108 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 17 - 0.00 seconds
Cgl0004I processed model has 16 rows, 26 columns (26 integer (23 of which binary)) and 78 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found o

In [27]:
import plotly.graph_objects as go


# 結果の可視化
is_legend_placed_lamps = True
is_legend_not_placed_lamps = True
is_plot_covered_points = True


fig = go.Figure()
for i, (pos, lamp, cover) in enumerate(zip(positions, lamp_status, point_covered)):
    if lamp == 1:
        fig.add_shape(
            type="circle",
            xref="x",
            yref="y",
            x0=pos[0] - coverage_range,
            y0=pos[1] - coverage_range,
            x1=pos[0] + coverage_range,
            y1=pos[1] + coverage_range,
            line_color="red",
            fillcolor="red",
            opacity=0.3,
            name="カバー範囲" if i == 0 else "",
            showlegend=False,
        )
        show_legend = is_legend_placed_lamps
        fig.add_trace(
            go.Scatter(
                x=[pos[0]],
                y=[pos[1]],
                mode="markers",
                marker=dict(color="blue", symbol="triangle-up", size=12),
                name="街灯（設置済み）",
                showlegend=show_legend,
            )
        )
        is_legend_placed_lamps = False
    elif cover == 0:
        show_legend = i not in mandatory_points and is_plot_covered_points
        fig.add_trace(
            go.Scatter(
                x=[pos[0]],
                y=[pos[1]],
                mode="markers",
                marker=dict(color="black", symbol="circle", size=10),
                name="カバーされていない位置"
                if i not in mandatory_points and is_plot_covered_points
                else "",
                showlegend=show_legend,
            )
        )
        is_plot_covered_points = False
    elif i in mandatory_points:
        show_legend = i == mandatory_points[0]
        fig.add_trace(
            go.Scatter(
                x=[pos[0]],
                y=[pos[1]],
                mode="markers",
                marker=dict(color="red", symbol="star", size=15),
                name="必須カバー位置",
                showlegend=show_legend,
            )
        )
    else:
        show_legend = is_legend_not_placed_lamps
        fig.add_trace(
            go.Scatter(
                x=[pos[0]],
                y=[pos[1]],
                mode="markers",
                marker=dict(color="green", symbol="square", size=12),
                name="カバーされた位置",
                showlegend=show_legend,
            )
        )
        is_legend_not_placed_lamps = False

# グラフ設定
fig.update_layout(
    title=f"街灯の設置問題とカバー範囲 - カバー率: {coverage_rate:.2f}%",
    xaxis_title="x座標",
    yaxis_title="y座標",
    legend_title="凡例",
    width=800,
    height=800,
    xaxis=dict(range=[-1, 12]),
    yaxis=dict(range=[-1, 12]),
)

fig.show()